In [1]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook

# Pycaret Library
from pycaret.regression import *

In [2]:
train_data = pd.read_csv('/Users/leeshinhaeng/Desktop/open/train.csv')
submit = pd.read_csv('/Users/leeshinhaeng/Desktop/open/sample_submission.csv')

In [3]:
train_data.head()

,ID,제품,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,0,0,0,1,3,2,0,0,2,0
2,2,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,B002-00002-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,B002-00003-00001,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# # Scaling 코드 (이희원님 코드 가져옴)

# # 숫자형 변수들의 min-max scaling을 수행하는 코드입니다.
# numeric_cols = train_data.columns[6:]
# # 칵 column의 min 및 max 계산
# min_values = train_data[numeric_cols].min(axis=1)
# max_values = train_data[numeric_cols].max(axis=1)
# # 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
# ranges = max_values - min_values
# ranges[ranges == 0] = 1
# # min-max scaling 수행
# train_data[numeric_cols] = (train_data[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)
# # max와 min 값을 dictionary 형태로 저장
# scale_min_dict = min_values.to_dict()
# scale_max_dict = max_values.to_dict()

In [5]:
# Label Encoding
label_encoder = LabelEncoder()
categorical_columns = ['제품','대분류', '중분류', '소분류', '브랜드']

# 범주형 칼럼별로 레이블 인코딩
for col in categorical_columns:
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])

In [6]:
train_data.head()

,ID,제품,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0,0,1,6,37,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,2,7,43,1,0,0,0,0,...,0,0,0,1,3,2,0,0,2,0
2,2,2,2,7,43,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,3,2,7,43,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,4,0,0,2,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
train_data.shape

(15890, 465)

In [8]:
# pip uninstall pycaret
# pip install pycaret[full]

In [9]:
# 일별 판매량 데이터 - train
train_split = train_data.drop(['제품','대분류','중분류','소분류','브랜드'], axis=1)
product_split = train_data[['ID','대분류','중분류','소분류','브랜드']]

In [25]:
# 기타 칼럼들을 ID와 날짜 칼럼에 해당되게끔 행으로 옮기는 작업 수행
# 이 경우 데이터가 700만건이 넘게 발생
melted_train = train_split.melt(id_vars=['ID'], var_name='Date', value_name='Value')

In [26]:
melted_train.head()

,ID,Date,Value
0,0,2022-01-01,0
1,1,2022-01-01,0
2,2,2022-01-01,0
3,3,2022-01-01,0
4,4,2022-01-01,0


In [27]:
melted_train['Date'] = pd.to_datetime(melted_train['Date'])
melted_train.sort_values(by=['ID','Date'], inplace=True)
melted_train.reset_index(drop=True, inplace=True)

In [28]:
# 데이터 합치기
melted_train = pd.merge(melted_train, product_split, on='ID')

In [29]:
product_split = train_data[['ID','대분류','중분류','소분류','브랜드']]

In [30]:
submit_sum = pd.merge(submit, product_split, on='ID')

In [31]:
submit2 = submit.melt(id_vars=['ID'], var_name='Date', value_name='Value')

In [32]:
submit_sum = pd.merge(submit2, product_split, on='ID')

In [33]:
submit_sum

,ID,Date,Value,대분류,중분류,소분류,브랜드
0,0,2023-04-05,0,1,6,37,0
1,0,2023-04-06,0,1,6,37,0
2,0,2023-04-07,0,1,6,37,0
3,0,2023-04-08,0,1,6,37,0
4,0,2023-04-09,0,1,6,37,0
...,...,...,...,...,...,...,...
333685,15889,2023-04-21,0,1,3,19,3169
333686,15889,2023-04-22,0,1,3,19,3169
333687,15889,2023-04-23,0,1,3,19,3169
333688,15889,2023-04-24,0,1,3,19,3169


In [35]:
from pycaret.regression import *

# # Date 칼럼을 Datetime 형태로 변환
# melted_train['Date'] = pd.to_datetime(melted_train['Date'])
# submit_sum['Date'] = pd.to_datetime(submit_sum['Date'])

# # Date 칼럼을 인덱스로 설정
# melted_train.set_index('Date', inplace=True)
# submit_sum.set_index('Date', inplace=True)



In [36]:
len(melted_train)

7293510

In [37]:
# 최근 120일 이전의 데이터를 제거하고 최근 120일 데이터만 남기기
recent_120_days = melted_train['Date'].iloc[-120]
melted_train = melted_train[melted_train['Date'] >= recent_120_days]

In [39]:
melted_train.reset_index(drop=True, inplace=True)

In [40]:
# pycaret 설정 및 데이터 로드
exp = setup(data=melted_train, session_id=123, fold_strategy='timeseries', target='Value', ignore_features=['대분류','브랜드','중분류','소분류'])

# 시계열 모델 생성 및 학습
best_model = compare_models(fold=5)

,Description,Value
0,Session id,123
1,Target,Value
2,Target type,Regression
3,Original data shape,"(1906800, 7)"
4,Transformed data shape,"(1906800, 5)"
5,Transformed train set shape,"(1334760, 5)"
6,Transformed test set shape,"(572040, 5)"
7,Ignore features,4
8,Numeric features,1
9,Date features,1


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,7.5519,7411.1679,84.4242,0.6520,0.8227,1.1017,21.6880
xgboost,Extreme Gradient Boosting,14.3453,9503.4327,96.8590,0.5490,1.7716,2.2287,8.2400
dt,Decision Tree Regressor,7.3679,10824.4987,102.5434,0.4767,0.7630,0.9962,0.6080
et,Extra Trees Regressor,13.8454,15770.3204,123.6060,0.2619,1.1956,2.6630,17.1600
gbr,Gradient Boosting Regressor,18.7834,15942.2349,125.8643,0.2403,2.0129,3.3301,11.0800
knn,K Neighbors Regressor,15.5105,17003.1434,129.6826,0.1928,1.3086,2.6236,0.6580
lightgbm,Light Gradient Boosting Machine,20.3230,20535.4929,142.9539,0.0189,2.0082,3.6583,1.1580
br,Bayesian Ridge,20.7489,20883.3120,144.1712,0.0020,2.2109,3.7853,0.4100
lr,Linear Regression,20.7502,20883.3237,144.1713,0.0020,2.2102,3.7903,0.5960
lar,Least Angle Regression,20.7502,20883.3237,144.1713,0.0020,2.2102,3.7903,0.3500


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [42]:
submit_sum

,ID,Date,Value,대분류,중분류,소분류,브랜드
0,0,2023-04-05,0,1,6,37,0
1,0,2023-04-06,0,1,6,37,0
2,0,2023-04-07,0,1,6,37,0
3,0,2023-04-08,0,1,6,37,0
4,0,2023-04-09,0,1,6,37,0
...,...,...,...,...,...,...,...
333685,15889,2023-04-21,0,1,3,19,3169
333686,15889,2023-04-22,0,1,3,19,3169
333687,15889,2023-04-23,0,1,3,19,3169
333688,15889,2023-04-24,0,1,3,19,3169


In [43]:
submit_sum['Date'] = pd.to_datetime(submit_sum['Date'])

# 예측 데이터 생성
predictions = predict_model(best_model, data=submit_sum)

# 예측 결과를 submit_sum의 Value 열에 저장
submit_sum['Value'] = predictions['prediction_label']

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,9.9060,5737.6606,75.7473,0.0000,1.5190,nan


In [47]:
predictions['prediction_label'].max()

4203.41

In [49]:
# 예측 결과를 submit_sum의 Value 열에 저장
submit_sum['Value'] = predictions['prediction_label']

In [52]:
submit_sum

,ID,Date,Value,대분류,중분류,소분류,브랜드
0,0,2023-04-05,0.03,1,6,37,0
1,0,2023-04-06,0.03,1,6,37,0
2,0,2023-04-07,0.03,1,6,37,0
3,0,2023-04-08,0.03,1,6,37,0
4,0,2023-04-09,0.03,1,6,37,0
...,...,...,...,...,...,...,...
333685,15889,2023-04-21,0.46,1,3,19,3169
333686,15889,2023-04-22,0.46,1,3,19,3169
333687,15889,2023-04-23,0.46,1,3,19,3169
333688,15889,2023-04-24,0.46,1,3,19,3169


In [51]:
submit_sum.pivot_table

0         0.03
1         0.03
2         0.03
3         0.03
4         0.03
          ... 
333685    0.46
333686    0.46
333687    0.46
333688    0.46
333689    0.46
Name: Value, Length: 333690, dtype: float64

In [54]:
# 피벗테이블 생성
pivot_submit = submit_sum.pivot_table(index='ID', columns='Date', values='Value', fill_value=0)


In [58]:
pivot_submit.to_csv()

Date,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,2023-04-14,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
ID,,,,,,,,,,,,,,,,,,,,,
0,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
1,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
2,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
3,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
4,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
15886,0.74,0.74,0.74,0.74,0.74,0.74,0.74,0.74,0.74,0.74,...,0.74,0.74,0.74,0.74,0.74,0.74,0.74,0.74,0.74,0.74
15887,0.26,0.26,0.26,0.26,0.26,0.26,0.26,0.26,0.26,0.26,...,0.26,0.26,0.26,0.26,0.26,0.26,0.26,0.26,0.26,0.26


In [194]:
submit

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15886,15886,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15887,15887,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15888,15888,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
for date in pivot_submit.columns[:]:
    submit[date] = pivot_submit[date]


In [72]:
pivot_submit.to_csv('pycaret1차(잘못됨).csv', encoding='utf-8-sig', index=True)

In [70]:
pivot_submit.reset_index(drop=False, inplace=True)

In [71]:
pivot_submit

Date,ID,2023-04-05 00:00:00,2023-04-06 00:00:00,2023-04-07 00:00:00,2023-04-08 00:00:00,2023-04-09 00:00:00,2023-04-10 00:00:00,2023-04-11 00:00:00,2023-04-12 00:00:00,2023-04-13 00:00:00,...,2023-04-16 00:00:00,2023-04-17 00:00:00,2023-04-18 00:00:00,2023-04-19 00:00:00,2023-04-20 00:00:00,2023-04-21 00:00:00,2023-04-22 00:00:00,2023-04-23 00:00:00,2023-04-24 00:00:00,2023-04-25 00:00:00
0,0,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
1,1,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
2,2,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
3,3,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
4,4,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
15886,15886,0.74,0.74,0.74,0.74,0.74,0.74,0.74,0.74,0.74,...,0.74,0.74,0.74,0.74,0.74,0.74,0.74,0.74,0.74,0.74
15887,15887,0.26,0.26,0.26,0.26,0.26,0.26,0.26,0.26,0.26,...,0.26,0.26,0.26,0.26,0.26,0.26,0.26,0.26,0.26,0.26
15888,15888,1.26,1.26,1.26,1.26,1.26,1.26,1.26,1.26,1.26,...,1.26,1.26,1.26,1.26,1.26,1.26,1.26,1.26,1.26,1.26


In [61]:
submit_sum.update(pivot_submit.iloc[:, 1:])

In [62]:
submit_sum

,ID,Date,Value,대분류,중분류,소분류,브랜드
0,0,2023-04-05,0.03,1,6,37,0
1,0,2023-04-06,0.03,1,6,37,0
2,0,2023-04-07,0.03,1,6,37,0
3,0,2023-04-08,0.03,1,6,37,0
4,0,2023-04-09,0.03,1,6,37,0
...,...,...,...,...,...,...,...
333685,15889,2023-04-21,0.46,1,3,19,3169
333686,15889,2023-04-22,0.46,1,3,19,3169
333687,15889,2023-04-23,0.46,1,3,19,3169
333688,15889,2023-04-24,0.46,1,3,19,3169
